In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!git clone https://github.com/ks-1180/art-style-challenge.git
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

Cloning into 'art-style-challenge'...
remote: Enumerating objects: 318, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 318 (delta 64), reused 153 (delta 53), pack-reused 154
Receiving objects: 100% (318/318), 145.68 MiB | 15.78 MiB/s, done.
Resolving deltas: 100% (126/126), done.
Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 128, done.
remote: Total 128 (delta 0), reused 0 (delta 0), pack-reused 128
Receiving objects: 100% (128/128), 1.12 MiB | 4.70 MiB/s, done.
Resolving deltas: 100% (57/57), done.


need this for styleGAN image generation for plugins: "bias_act_plugin" and "upfirdn2d_plugin"

In [ ]:
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force
!rm ninja-linux.zip

In [ ]:
!pip install gradio

In [10]:
os.path.exists('/content/stylegan2-ada-pytorch')

True

In [1]:
import gradio as gr
import os
import sys
# TODO: change to repo after finishing
%cd /content/stylegan2-ada-pytorch

library_paths = ['.']

import dnnlib, legacy

for p in library_paths:
    if p not in sys.path:
        sys.path.append(p)

# pls update root accordingly
%cd /content/art-style-challenge/src

#from art-style-challenge import evaluator
import generator
import projector

import matplotlib.pyplot as plt
import cv2
import PIL.Image
import torch

/content/stylegan2-ada-pytorch
/content/art-style-challenge/src


# Setup

In [2]:
resolution = 256
base_model = 'https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/transfer-learning-source-nets/ffhq-res256-mirror-paper256-noaug.pkl'
style_models_f = 'https://drive.google.com/drive/folders/1BqbgoMRUAmooIn2lMWO9K4x-EBAekYry?usp=sharing'
style_models_m = 'https://drive.google.com/drive/folders/1u0XLGhcJV8txMS8Ho_R9Jr09hztZ2vsL?usp=sharing'

#gen = generator.Generator(base_model, style_models_f, resolution) # takes time to generate all models 

In [3]:
gen_female = generator.Generator(base_model, style_models_f, resolution)

In [4]:
gen_male = generator.Generator(base_model, style_models_m, resolution)

execute once to setup: "bias_act_plugin" and "upfirdn2d_plugin"

# Gradio App

In [ ]:
#from IPython.lib.security import passwd
import numpy as np
from PIL import Image
import time

def gen_image(image, features, iterations, dropdown):

  #for _ in range(2):
  #    time.sleep(1)
  #    image = np.random.random((600, 600, 3))
  #    yield image

  #image = "https://i.picsum.photos/id/867/600/600.jpg?hmac=qE7QFJwLmlE_WKI7zMH6SgH5iY5fx8ec6ZJQBwKRT44"
  #yield image
  
  gen = gen_female if features == 'feminine' else gen_male
  append = '_female.pkl' if features == 'feminine' else '_male.pkl'
  styles = [s + append for s in dropdown]

  if image is None:
    img, seed = gen.generate_seed()
    #images = gen.generate_art_styles(seed, styles)
    yield img

  else:

    p = projector.Projector(gen.G_ffhq)
    latent = p.run_projection(image, iterations)
    s = []
    for seed in latent:
      s = seed.cpu().numpy()
      img = gen.generate_latent(s, gen.G_ffhq)
      yield img

    styles = gen.generate_art_styles_w(s, styles)
    yield styles

    
    

demo = gr.Interface(
    fn=gen_image,
    inputs=[
        gr.Image(type="pil"),
        gr.Radio(["feminine", "masculine"], value="feminine"),
        gr.Slider(1, 1000, label='amount of iterations', value=500),
        gr.Dropdown(
            ['digital_art', 'fire_emblem', 'naruto', 'one_piece', 'painting', 'ghibli'], 
            label=['Choose your art styles'], 
            value=['digital_art', 'fire_emblem', 'naruto', 'one_piece', 'painting', 'ghibli'], multiselect=True)
    ],
    outputs=[
        "image"
    ]
)

demo.queue(concurrency_count=5, max_size=20)

demo.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://7619ab54-7d85-4de8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
tensor([[ 7.5837e-01, -4.8657e-01,  1.8486e+00,  4.9817e-01,  2.3777e+00,
         -1.7957e-02,  4.1466e-01, -1.7598e+00, -4.0464e-01,  8.9863e-01,
          2.7395e-01, -1.1658e+00, -8.1561e-01,  1.1487e+00,  5.9599e-01,
          8.3202e-02, -9.3373e-01, -1.3952e+00, -1.8714e+00,  1.7546e+00,
         -1.8251e+00,  3.2387e-02, -1.0230e+00,  1.0587e+00, -3.7423e-02,
         -6.0173e-01, -7.7683e-01,  4.1568e-01, -6.4208e-01,  1.4869e+00,
          1.2130e+00, -1.9579e-01, -4.9821e-01,  1.0909e+00, -2.2663e+00,
         -2.7130e-01, -7.1621e-01, -1.1598e+00, -9.8136e-02, -4.0880e-01,
          6.9870e-02, -2.2657e-02,  7.7984e-02, -1.1417e-01,  8.1118e-01,
         -5.1498e-01,  1.0449e+00, -5.2566e-01,  8.7448e-01,  1.5367e-02,
          8.3993e-01,  4.5643e-01, -2.7654e-01,  4.7952e-01,  1.8641e-02,
          2.2540e-01, -3.8882e-01,  1.3359e+00,  3.2054e-01,  2.2117e-01,
     

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/gradio/routes.py", line 337, in run_predict
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.8/dist-packages/gradio/blocks.py", line 1015, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.8/dist-packages/gradio/blocks.py", line 847, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.8/dist-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.8/dist-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.8/dist-packages/anyio/_backends/_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "/usr/local/lib/python3.8/dist-packages/gradio/utils.py", line 410, in async_iteration
    return next(iterator)
  File "<ipy

Keyboard interruption in main thread... closing server.
